Ablauf:
Fiel "AQM_import_API"
1. Import Data via API from Refinitiv to Jupiter Notebook
2. Create DataFrame, check Data and clean Data if needed:
    - N/A
    - Sprünge?
    - Format?
3. Save clean Data in SQL Lite in DB


File "AQM_Analyse":
1. Import the clean Data from SQL DB into Notebook
2. Prepare Data for Analysis
3. Create Dataframe to save the Results
4. Create Loop, which executes the Test (Autocorrelation) for each Instrument (Future)
5. Save the Results from the Loop into the "Result"-Dataframe
6. Export Results to SQL Lite into Table "Results"

In [4]:
import pandas as pd
import numpy as np

import scipy
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS

import math
from math import exp, sqrt, log

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import seaborn as sns
import xlsxwriter

import datetime

import eikon as ek
creds = 'd505f265d9bd4c0cb7df44481ece31a0ad8d1974'
#API Key Lionel: d505f265d9bd4c0cb7df44481ece31a0ad8d1974
#API Key Raphi:
#API Key Laurin:
ek.set_app_key(creds)


2022-03-21 14:23:00,538 P[7108] [MainThread 23612] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2022-03-21 14:23:00,540 P[7108] [MainThread 23612] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2022-03-21 14:23:00,541 P[7108] [MainThread 23612] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2022-03-21 14:23:00,542 P[7108] [MainThread 23612] Port number was not identified, cannot send any request


In [ ]:
#Define the Future Contracts

Futures = ['LCOc1',     #ICE Europe Brent Crude 
           'CLc1',      #NYMEX Light Sweet Crude Oil (WTI)
           'NGLNMc1',   #Intercontinetanl Exchange UK NBP Natrual Gas
           'NGc1',      #NYMEX Henry Hub Natural Gas
           'GCc1',      #COMEX Gold
           'SIc1',      #COMEX Silver
           'PLc1',      #NYM Platinum
           'PAc1',      #CYM Palladium
           'Wc1',       #CBoT Wheat
           'Cc1',       #CBoT Corn
           'LCc1',      #CME Live Cattle
           'OJc1'       #ICE Orange Juice
          ]

In [ ]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime

# Here you can put your key; I read from a local file to avoid sharing password
creds = 'd505f265d9bd4c0cb7df44481ece31a0ad8d1974'
ek.set_app_key(creds)
#import refinitiv.dataplatform.eikon as ek
ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')
df, err = ek.get_data(
    instruments =  ['LCOc1', 'CLc1',  'NGLNMc1'],
    fields = ['TR.CLOSEPRICE'],
    parameters = {
        'SDate': '2000-01-01',
        'EDate': '2022-01-01',
        'Frq': 'D'
    }
)

display(df)

In [ ]:
#Get the Close Prices of the futures



data = ek.get_timeseries(Futures,
                        start_date = '2012-01-01',
                        end_date = '2022-03-18',
                        fields = 'CLOSE',
                        interval = 'daily'
                        )

#data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d').dt.date # convert format to %Y-%m-%d

data

In [ ]:
# of Investors of AAPL -- this is a real-world case based on our work
# Step 0 : Fire up Refinitiv Workspace
# Step 1a : Get code for stock AAPL.O
# Step 1b : Use Data-Broswer to find series code
# (Show data filter and browser in the tap)
# look at Apple and no dice
# no dice : higher level at Filters : key-word investor
# Step one Import the libraries
# this is the Eikon library, almost all APIs have this type of class that allows one to Specify how to interact with the API
import eikon as ek
import pandas as pd
import numpy as np
import datetime

# Here you can put your key; I read from a local file to avoid sharing password
creds = 'mykey'
ek.set_app_key(creds)

# Define a wrapper function (not strickly necessary)
def holding_data(instrument='LCOc1', fields=[], start_year=2019, end_year=2020):
    fields = ["CLOSE"]
    dat, err = ek.get_data('LCOc1',
                           fields = 'CLOSE',
                           parameters={'SDate': '{}-01-01'.format(2019),
                                       'EDate': '{}-12-31'.format(2020),
                                       'Frq':'D'}
                           )


    dat['Date'] = pd.to_datetime(dat['Date'], format='%Y-%m-%d').dt.date # convert format to %Y-%m-%d

    dat = dat.sort_index() # sort Date in ascending order
    # data base containts weak data points: drop na's
    dat = dat.dropna() # I'm ambivalent about this...

    return(dat)

# Assigns the return of the function to a variable
dat = holding_data()
# Writes the data frame to the CSV
dat.to_csv('commodities.csv')
print(dat.head(10))

In [ ]:
def holding_data(instrument='LCOc1', fields=[], start_year=2019, end_year=2020):
    fields = ['CLOSE']
dat, err = ek.get_data('AAPL.O',
                       fields = fields,
                       parameters={'SDate': '{}-01-01'.format(2019),
                                   'EDate': '{}-12-31'.format(2020),
                                   'Frq':'Q'}
                       )


In [ ]:
data.info()

In [ ]:
data.plot(subplots = True, figsize = (15,10));

In [ ]:
df = ek.get_timeseries(["LCOc1"], 
                       start_date="2022-03-10",  
                       end_date="2022-03-18")
df

In [ ]:
df, err = ek.get_data(['LCOc1', 'CLc1'], 
                      ['CF_CLOSE'],
                      {'Scale': 6, 'SDate': 0, 'EDate': -2, 'FRQ': 'FY', 'Curn': 'USD'})
df

In [ ]:
df, err = ek.get_data(['LCOc1', 'CLc1'], 
                      ['CF_CLOSE'],
                      {'Scale': 6, 'SDate': 0, 'EDate': -2, 'FRQ': 'FY', 'Curn': 'EUR'})
df

In [ ]:
def holding_data(instrument='AAPL.O', fields=[], start_year=2019, end_year=2020):
    fields = ['CF_CLOSE']
    


In [ ]:
dat, err = ek.get_data('AAPL.O',
                       fields = fields,
                       parameters={'SDate': '{}-01-01'.format(2019),
                                   'EDate': '{}-12-31'.format(2020),
                                   'Frq':'D'}
                       )


dat['Date'] = pd.to_datetime(dat['Date'], format='%Y-%m-%d').dt.date # convert format to %Y-%m-%d

dat = dat.sort_index() # sort Date in ascending order
# data base containts weak data points: drop na's
dat = dat.dropna() # I'm ambivalent about this...

return(dat)

Save Data in SQL Database (DB)

In [ ]:
#already created a DB in SQL Lite
#insert the clean Data in the SQL DB: "SQL_DB" into the Table "REFENITIV_DATA":

In [ ]:
import sqlite3
from sqlite3 import Error
 
""" create a database connection tool to a SQLite database """
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()
 
 
if __name__ == '__main__':
    create_connection(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")

In [ ]:
conn = sqlite3.connect(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")

#Add single Data in Table:
conn.execute("INSERT INTO REFINITIV_DATA(ID,NAME,AGE,ADDRESS,SALARY) VALUES (1, 'Paul', 32, 'California', 20000.00)");
conn.commit()

#Add all Data at once:
c = conn.cursor()
c.executemany("INSERT INTO Inflation_data(CPIAUCSL,WTISPLC,T5YIFRM,DATE) VALUES (?,?,?,?)", records_to_insert)    
conn.commit()

In [ ]:
#Let's now select the data from the database and pull them to python 
c = conn.cursor()
c.execute("SELECT * FROM REFINITIV_DATA")
#c.execute("SELECT * FROM INSTRUMENT")
#c.execute("SELECT * FROM RESULTS")
new_data=c.fetchall()
new_data